# Data augmentation

Het verzamelen van voldoende data kan zeer tijdrovend en kostelijk zijn.
Daarom zou het handig zijn om op een automatische manier extra data aan te maken.
Dit is wat data augmentation doet en wordt vaak gebruikt wanneer er gewerkt wordt met visuele of auditieve data.
De code hieronder zal vooral een voorbeeld zijn van hoe je data augmentatie kan toepassen op beelden/images.
Hierbij gaan we voor het trainen op een random manier een variatie maken van de figuur.
Doordat er meerdere epochs gebruikt worden bij training zullen er op die manier meerdere varianten van elke figuur gebruikt worden bij training.

Data augmentation kan je meestal toepassen zonder problemen. Zeker de opties waarbij geen pixels verdwijnen geven normaal geen issues.
Let echter wel op als je roteert of cropt en hetgene dat je moet detecteren aan de rand van je scherm ligt, dat dat object kan verdwijnen.
Indien je genoeg epochs gebruikt zou dit echter geen groot probleem mogen zijn door compensatie in de andere epochs.

In de vorige notebook hadden we onderstaande model om een classificatie CNN te maken. 

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Stap 1: Dataset Inladen en Voorbereiden
transform = transforms.Compose([ 
    transforms.ToTensor(),  # Zet de afbeelding om naar een tensor en normaliseer naar [0, 1]
])

# Laad de CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Define the CNN model using torch.nn.Sequential
model = nn.Sequential(
    nn.Conv2d(3, 6, 5),       # Convolutional layer: input channels=3, output channels=6, kernel size=5
    nn.ReLU(),                # ReLU activation
    nn.MaxPool2d(2, 2),       # Max pooling layer: kernel size=2, stride=2
    nn.Conv2d(6, 16, 5),      # Convolutional layer: input channels=6, output channels=16, kernel size=5
    nn.ReLU(),                # ReLU activation
    nn.MaxPool2d(2, 2),       # Max pooling layer: kernel size=2, stride=2
    nn.Flatten(),             # Flatten the output for the fully connected layers
    nn.Linear(16 * 5 * 5, 120), # Fully connected layer: input features=16*5*5, output features=120
    nn.ReLU(),                # ReLU activation
    nn.Linear(120, 84),       # Fully connected layer: input features=120, output features=84
    nn.ReLU(),                # ReLU activation
    nn.Linear(84, 10)         # Output layer: input features=84, output features=10 (number of classes)
).to(device)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(2):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

# Evaluate the model
dataiter = iter(testloader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)

# Make predictions
outputs = model(images)
_, predicted = torch.max(outputs, 1)

# Print the image, true labels, and predicted labels
def imshow(img):
    img = img.permute(1, 2, 0).numpy()  # Convert back to HxWxC
    plt.imshow(img)
    plt.show()

# Print images
imshow(torchvision.utils.make_grid(images.cpu()))
print('GroundTruth: ', ' '.join(f'{trainset.classes[labels[j]]:5s}' for j in range(4)))
print('Predicted:  ', ' '.join(f'{trainset.classes[predicted[j]]:5s}' for j in range(4)))

Om op de cifar10 dataset data augmentation uit te voeren moeten we enkel de transform stappenplan aanpassen om op een random manier bewerkingen uit te voeren op de figuren.
We kunnen bijvoorbeeld de transform aanpassen als volgt:

In [1]:
transform = transforms.Compose([ 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),  # Zet de afbeelding om naar een tensor en normaliseer naar [0, 1]
])

NameError: name 'transforms' is not defined

## Keras

De analoge oplossing om dit te doen met keras is door het model in onderstaande voorbeeld

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras_core as keras
from keras_core import Sequential
from keras_core import layers

# Laad en bereid de CIFAR-10 dataset voor
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Min-Max scaling naar [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Definieer het CNN-model met tf.keras.Sequential
model = Sequential([
    layers.Rescaling(1.0/255.0),
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile het model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train het model
history = model.fit(x_train, y_train, epochs=10, batch_size=64,
                    validation_split=0.1, verbose=2)

# Evalueer het model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')

# Maak voorspellingen
predictions = model.predict(x_test)

aan te passen op de volgende manier. 

In [ ]:
model = models.Sequential([
    # Data Augmentation Layers
    layers.RandomFlip("horizontal"),          # Willekeurig horizontaal flippen van de afbeeldingen
    layers.RandomRotation(0.1),               # Willekeurig roteren van de afbeeldingen met maximaal 10%
    layers.RandomZoom(0.1),                   # Willekeurig zoomen in de afbeeldingen met maximaal 10%
    layers.RandomContrast(0.1),               # Willekeurig aanpassen van het contrast met maximaal 10%
    
    # Convolutionele Lagen
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])


# Oefening: Data Augmentation met Keras Core

In deze oefening gaan we het concept van **data augmentation** oefenen. Je gebruikt de **CIFAR-10 dataset** en probeert verschillende augmentaties toe te passen op de trainingsbeelden. 
Voer de volgende stappen uit:

* Laad de CIFAR-10 dataset
* Definieer een aantal data augmentation layers
  - Rotatie van de afbeeldingen: willekeurige rotatie tussen -15 graden en +20 graden
  - Horizontale flip: afbeeldingen af en toe horizontaal spiegelen (kans van 30%)
  - Translatie in hoogte en breedte: verschuiving tot 10% van de hoogte en 10% van de breedte
  - Variatie in helderheid: helderheid aanpassen met een factor tussen 0.8 en 1.2
  - Pas een contrast-aanpassing toe zodat sommige beelden iets hoger of lager contrast hebben
* Pas de augmentaties toe op een willekeurige batch en toon 10 figuren voor en na augmentatie
  
**Extra opdrachten**

* Experimenteer met verschillende waarden voor de parameters van de augmentaties.
* Voeg meer augmentaties toe zoals RandomZoom, RandomCrop, RandomContrast (eventueel met een custom layer).
* Observeer hoe de augmented afbeeldingen eruit zien en noteer welke augmentaties het meest effectief lijken voor training van een CNN.
* Wat is het effect van de data augmentatie op de prestaties van het bovenstaande model